In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
DROP Table IF EXISTS Person;
DROP Table IF EXISTS Drug;
DROP Table IF EXISTS DrugUse;

In [3]:
CREATE TABLE Person (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/demographic-person.csv');

CREATE TABLE Drug (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(100) NOT NULL,
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/medications-drug.csv');

CREATE TABLE DrugUse (
  PersonId VARCHAR(5) NOT NULL,
  DrugCode VARCHAR(6) NOT NULL,
  DaysUse INTEGER,
  FOREIGN KEY(PersonId)
    REFERENCES Person(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(DrugCode)
    REFERENCES Drug(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/medications-use.csv');

In [4]:
SELECT * FROM Person;
SELECT * FROM Drug;
SELECT * FROM DrugUse

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [5]:
SELECT DR.name, COUNT(*) QUANTIDADE
FROM Person PE, Drug DR, DrugUse DU
WHERE PE.id = DU.personid AND DU.drugcode = DR.code
GROUP BY DR.name

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [6]:
DROP Table IF EXISTS MedMaisUsado;

-- MED USADO POR MAIS DIAS (TOTAL DE PESSOAS)
CREATE VIEW MedMaisUsado AS
SELECT DU.DrugCode, SUM(DaysUse) QUANTIDADE_DIAS
FROM DrugUse DU
GROUP BY DU.DrugCode
ORDER BY QUANTIDADE_DIAS;

SELECT *
FROM MedMaisUsado;

In [7]:
SELECT M.DrugCode
FROM MedMaisUsado M
WHERE M.QUANTIDADE_DIAS = SELECT MAX(Q.QUANTIDADE_DIAS)
FROM MedMaisUsado Q;

d00278

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [8]:
SELECT DU3.DrugCode -- seleciona o que ta fora do grupo
    FROM DrugUse DU3
    GROUP BY DU3.DrugCode
    HAVING SUM(DU3.DaysUse) NOT IN (SELECT SUM(DU2.DaysUse) QUANTIDADE_DIAS -- tira o maior
    FROM DrugUse DU2
    GROUP BY DU2.DrugCode
    HAVING QUANTIDADE_DIAS < ANY (SELECT SUM(DU1.DaysUse) -- Seleciona somas de uso de cada medicamento
    FROM DrugUse DU1
    GROUP BY DU1.DrugCode));

d00278

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [9]:
SELECT D.Name -- seleciona o que ta fora do grupo
    FROM DrugUse DU3, Drug D
    WHERE DU3.DrugCode = D.Code
    GROUP BY DU3.DrugCode
    HAVING SUM(DU3.DaysUse) NOT IN (SELECT SUM(DU2.DaysUse) QUANTIDADE_DIAS -- tira o maior
    FROM DrugUse DU2
    GROUP BY DU2.DrugCode
    HAVING QUANTIDADE_DIAS < ANY (SELECT SUM(DU1.DaysUse) -- Seleciona somas de uso de cada medicamento
    FROM DrugUse DU1
    GROUP BY DU1.DrugCode));

LEVOTHYROXINE

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [10]:
SELECT PE.id, AVG(DU.daysuse) MEDIA_DE_USO
FROM Person PE, Drug DR, DrugUse DU 
WHERE PE.id = DU.personid AND DU.drugcode = DR.code
GROUP BY PE.id
ORDER BY PE.id;

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [11]:
DROP Table IF EXISTS MEDIAMEDPERPERSON;
DROP Table IF EXISTS MEDPERPERSON;

CREATE VIEW MEDPERPERSON AS
SELECT PE.id, COUNT(*) QUANT_MED
    FROM Person PE, Drug DR, DrugUse DU
    WHERE PE.id = DU.personid AND DU.drugcode = DR.code
    GROUP BY PE.id;

CREATE VIEW MEDIAMEDPERPERSON AS 
SELECT AVG(MMP.QUANT_MED) MEDIA
FROM MEDPERPERSON MMP;

SELECT MEDIA
FROM MEDIAMEDPERPERSON;

3

In [12]:
SELECT MMP.ID, MMP.QUANT_MED
FROM MEDPERPERSON MMP, MEDIAMEDPERPERSON MMMP
WHERE MMP.QUANT_MED > MMMP.MEDIA
ORDER BY MMP.ID;

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [13]:
SELECT P.age, AVG(MMP.QUANT_MED) MEDIA_MED_POR_IDADE
FROM Person P, MEDPERPERSON MMP
WHERE P.id = MMP.id
GROUP BY P.age;

# Navegando por Hierarquias

A seguir exercícios envolvendo navegação por hierarquias.

# Marcadores e Taxonomia

Considere o modelo para `Marcadores` e `Taxonomia`:

![UML](marcadores-taxonomia-uml.png)

![Relacional](marcadores-taxonomia-er.png)

Considere as sentenças SQL a seguir para montar as tabelas de `Marcadores` e `Taxonomia`:

In [14]:
DROP TABLE IF EXISTS Taxonomia;
DROP TABLE IF EXISTS Marcadores;

CREATE TABLE Taxonomia (
  Categoria VARCHAR(50) NOT NULL,
  Superior  VARCHAR(50),
  PRIMARY KEY(Categoria),
  FOREIGN KEY(Superior)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

CREATE TABLE Marcadores (
  Titulo VARCHAR(50) NOT NULL,
  Endereco VARCHAR(80) NOT NULL,
  Acessos INTEGER,
  Categoria VARCHAR(50) NOT NULL,
  PRIMARY KEY(Titulo),
  FOREIGN KEY(Categoria)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

As tabelas são preenchidas com as seguintes tuplas:

In [15]:
INSERT INTO Taxonomia VALUES ('Geral', NULL);
INSERT INTO Taxonomia VALUES ('Serviços', 'Geral');
INSERT INTO Taxonomia VALUES ('Acadêmico', 'Geral');
INSERT INTO Taxonomia VALUES ('Relacionamento', 'Geral');
INSERT INTO Taxonomia VALUES ('Busca', 'Serviços');
INSERT INTO Taxonomia VALUES ('Portal', 'Serviços');
INSERT INTO Taxonomia VALUES ('Mail', 'Serviços');
INSERT INTO Taxonomia VALUES ('Vendas', 'Serviços');
INSERT INTO Taxonomia VALUES ('Universidade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('CG', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Sociedade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Rede Social', 'Relacionamento');
INSERT INTO Taxonomia VALUES ('Marketplace', 'Vendas');
INSERT INTO Taxonomia VALUES ('Loja', 'Vendas');
INSERT INTO Taxonomia VALUES ('Notícias', 'Portal');
INSERT INTO Taxonomia VALUES ('Agregador', 'Portal');

INSERT INTO Marcadores VALUES ('Terra', 'http://www.terra.com.br', 295, 'Portal');
INSERT INTO Marcadores VALUES ('POVRay', 'http://www.povray.org', 2, 'CG');
INSERT INTO Marcadores VALUES ('SBC', 'http://www.sbc.org.br', 26, 'Sociedade');
INSERT INTO Marcadores VALUES ('Correios', 'http://www.correios.com.br', 45, 'Serviços');
INSERT INTO Marcadores VALUES ('GMail', 'http://www.gmail.com', 296, 'Mail');
INSERT INTO Marcadores VALUES ('Google', 'http://www.google.com', 1590, 'Busca');
INSERT INTO Marcadores VALUES ('Yahoo', 'http://www.yahoo.com', 134, 'Serviços');
INSERT INTO Marcadores VALUES ('Orkut', 'http://www.orkut.com', 45, 'Rede Social');
INSERT INTO Marcadores VALUES ('iBahia', 'http://www.ibahia.com', 3, 'Portal');
INSERT INTO Marcadores VALUES ('Submarino', 'http://www.submarino.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Amazon', 'https://www.amazon.com.br', 410, 'Marketplace');
INSERT INTO Marcadores VALUES ('Americanas', 'https://www.americanas.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Mercado Livre', 'https://www.mercadolivre.com.br', 500, 'Marketplace');
INSERT INTO Marcadores VALUES ('G1', 'https://g1.globo.com', 1200, 'Notícias');
INSERT INTO Marcadores VALUES ('Folha', 'https://www.folha.uol.com.br', 850, 'Notícias');
INSERT INTO Marcadores VALUES ('Google News', 'https://news.google.com', 900, 'Agregador');
INSERT INTO Marcadores VALUES ('Flipboard', 'https://flipboard.com', 950, 'Agregador');

Conteúdo da tabela `Taxonomia`:

In [16]:
SELECT * FROM Taxonomia;

Conteúdo da tabela `Marcadores`:

In [17]:
SELECT * FROM Marcadores;

## Tarefa 1

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, sem considerar as categorias subordinadas.

In [18]:
SELECT M.titulo, M.categoria
FROM Marcadores M
WHERE M.categoria = 'Serviços';

## Tarefa 2

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, considerando as categorias subordinadas.

In [19]:
SELECT M.titulo, T.categoria, T.superior
FROM Marcadores M, Taxonomia T, Taxonomia TS
WHERE M.categoria = T.categoria AND T.superior = TS.categoria AND (T.categoria = 'Serviços' OR T.superior = 'Serviços' OR TS.superior = 'Serviços')
ORDER BY TS.superior, T.superior, T.categoria;